## Importing library

In [3]:
import os
import warnings
import sys

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

## Loading dataset.

In [16]:
data_path = "data\wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2184,6.4,0.330,0.24,9.8,0.041,29.0,109.0,0.99560,3.29,0.47,10.1,6
3533,6.6,0.220,0.30,14.7,0.045,50.0,136.0,0.99704,3.14,0.37,10.6,6
52,6.2,0.160,0.33,1.1,0.057,21.0,82.0,0.99100,3.32,0.46,10.9,7
2905,7.6,0.310,0.26,1.7,0.073,40.0,157.0,0.99380,3.10,0.46,9.8,5
2854,6.4,0.210,0.28,5.9,0.047,29.0,101.0,0.99278,3.15,0.40,11.0,6
3298,6.3,0.240,0.35,2.3,0.039,43.0,109.0,0.99056,3.34,0.44,11.8,6
2451,6.6,0.560,0.15,10.0,0.037,38.0,157.0,0.99642,3.28,0.52,9.4,5
207,10.2,0.440,0.88,6.2,0.049,20.0,124.0,0.99680,2.99,0.51,9.9,4
4835,5.6,0.295,0.26,1.1,0.035,40.0,102.0,0.99154,3.47,0.56,10.6,6
2088,8.7,0.150,0.30,1.6,0.046,29.0,130.0,0.99420,3.22,0.38,9.8,6


## 1. Tracking experiments

In [26]:
# mlflow server --backend-store-uri mlruns/ --default-artifact-root mlruns/ --host 127.0.0.1 --port 8080
remote_server_uri = "http://127.0.0.1:8080"
mlflow.set_tracking_uri(remote_server_uri)

In [27]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:8080'

In [29]:
exp_name = "ElasticNet_wine2"
mlflow.set_experiment(exp_name)

2024/08/09 11:54:51 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/431960331090441242', creation_time=1723184692484, experiment_id='431960331090441242', last_update_time=1723184692484, lifecycle_stage='active', name='ElasticNet_wine2', tags={}>

In [30]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "g:\My Drive\Training\TAMLEP\Module_3\Assignment_3_2\data\wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))
        
        mlflow.sklearn.log_model(lr, "model")

In [31]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
Save to: mlflow-artifacts:/431960331090441242/e9e7923e51a048a29354ee174eba6b10/artifacts


2024/08/09 11:55:46 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/09 11:55:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-elk-966 at: http://127.0.0.1:8080/#/experiments/431960331090441242/runs/e9e7923e51a048a29354ee174eba6b10.
2024/08/09 11:55:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/431960331090441242.


In [32]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.20224631822892092
Save to: mlflow-artifacts:/431960331090441242/9168962e7961404398a756ca5115a368/artifacts


2024/08/09 11:57:18 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/09 11:57:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run omniscient-loon-620 at: http://127.0.0.1:8080/#/experiments/431960331090441242/runs/9168962e7961404398a756ca5115a368.
2024/08/09 11:57:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/431960331090441242.


In [33]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118586
  R2: 0.2157063843066196
Save to: mlflow-artifacts:/431960331090441242/0b24fe9eb23b4b53a0c7cfbd8f0c71cf/artifacts


2024/08/09 11:58:00 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/08/09 11:58:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run incongruous-tern-201 at: http://127.0.0.1:8080/#/experiments/431960331090441242/runs/0b24fe9eb23b4b53a0c7cfbd8f0c71cf.
2024/08/09 11:58:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/431960331090441242.


## 2. Tagging runs

In [35]:
from datetime import datetime
from mlflow.tracking import MlflowClient

In [38]:
client = MlflowClient()
# experiments = mlflow.list_experiments() # returns a list of mlflow.entities.Experiment
# print(experiments)

In [39]:
# get the run
_run = client.get_run(run_id="e9e7923e51a048a29354ee174eba6b10")
print(_run)

<Run: data=<RunData: metrics={'mae': 0.6278761410160693, 'r2': 0.12678721972772689, 'rmse': 0.82224284975954}, params={'alpha': '0.5', 'l1_ratio': '0.5'}, tags={'mlflow.log-model.history': '[{"run_id": "e9e7923e51a048a29354ee174eba6b10", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-08-09 06:25:24.129994", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.12.3", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null

In [40]:
# add a tag to the run
dt = datetime.now().strftime("%d-%m-%Y (%H:%M:%S.%f)")
client.set_tag(_run.info.run_id, "deployed", dt)

## 3. Deploy the model

Deploy the model locally by running 

`mlflow models serve -m mlruns/0/e9e7923e51a048a29354ee174eba6b10/artifacts/model/ -h 127.0.0.1 -p 8080`

Test the endpoint:

`curl -X POST -H "Content-Type:application/json; format=pandas-split" --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}' http://127.0.0.1:8080/invocations`